## TRDE703 Atelier Intégration des Données

In [1]:
import sys
import os
from pathlib import Path

current_dir = Path(os.getcwd())
project_root = current_dir.parent if current_dir.name == "etl" else current_dir

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"✅ Racine ajoutée au path : {project_root}")

from pyspark.sql import SparkSession
from etl.shared.config import SPARK_CONFIG

%load_ext autoreload
%autoreload 2

builder = SparkSession.builder
for key, val in SPARK_CONFIG.items():
    builder = builder.config(key, val)

spark = builder.getOrCreate()

print(f"✅ Session Spark créée avec le JAR : {SPARK_CONFIG.get('spark.jars')}")

✅ Racine ajoutée au path : /Users/cedricsanchez/Master1/Cours/integration_donnees_TP


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/20 09:36:14 WARN Utils: Your hostname, MacBook-Pro-de-Cedric.local, resolves to a loopback address: 127.0.0.1; using 10.101.116.72 instead (on interface en0)
26/01/20 09:36:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
26/01/20 09:36:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ Session Spark créée avec le JAR : /Users/cedricsanchez/Master1/Cours/integration_donnees_TP/jars/mysql-connector-j-9.1.0.jar


In [2]:
json_filepath = str(project_root / "data" / "raw" / "openfoodfacts-products.jsonl")

print(f"📂 Fichier cible : {json_filepath}")

if os.path.exists(json_filepath):
    print("✅ Le fichier existe bien.")
else:
    print("❌ Fichier introuvable. Vérifie le dossier data/raw/")

📂 Fichier cible : /Users/cedricsanchez/Master1/Cours/integration_donnees_TP/data/raw/openfoodfacts-products.jsonl
✅ Le fichier existe bien.


### 💡 Un mot sur nos choix (et les consignes du TP)

Pourquoi s'embêter à écrire ce schéma manuellement ?

1.  **Respect de la consigne :** Le sujet est strict : *"Lecture JSON/CSV avec schéma explicite (pas d'inférence magique en prod)"*. Utiliser `inferSchema=True` nous ferait perdre des points.
2.  **Gestion de l'Historique (SCD2) :** Le sujet impose de gérer le *"SCD2 produit"*. Pour cela, nous avons impérativement besoin du timestamp brut (`last_modified_t` en `LongType`) pour comparer les versions à la seconde près.
3.  **Structure Imbriquée :** Le format JSONL groupe les nutriments dans un objet. Notre schéma reflète cette réalité (`StructType` imbriqué) pour éviter de créer 1000 colonnes plates inutiles.
4.  **Sécurité (`String`) :** Pour des champs instables comme `nova_group`, on lit en `String` pour éviter que Spark ne rejette la ligne en cas de format inattendu.

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, LongType, ArrayType

def get_jsonl_schema():
    """
    Schéma robuste pour l'ingestion JSONL.
    Gère les types imbriqués (nested) propres à MongoDB/JSON.
    """

    nutriments_schema = StructType([
        StructField("energy-kcal_100g", FloatType(), True),
        StructField("sugars_100g", FloatType(), True),
        StructField("salt_100g", FloatType(), True),
        StructField("sodium_100g", FloatType(), True),
        StructField("fiber_100g", FloatType(), True),
        StructField("proteins_100g", FloatType(), True)
    ])

    return StructType([
        # --- Identifiants & Métadonnées ---
        StructField("code", StringType(), True),
        StructField("product_name", StringType(), True),
        StructField("last_modified_t", LongType(), True),
        StructField("created_t", LongType(), True),

        # --- Dimensions (Marques, Catégories...) ---
        StructField("brands", StringType(), True),
        StructField("categories", StringType(), True),
        StructField("countries_tags", ArrayType(StringType()), True),

        # --- Qualité & Scores ---
        StructField("nutriscore_grade", StringType(), True),
        StructField("nova_group", IntegerType(), True),
        StructField("ecoscore_grade", StringType(), True),

        # --- Mesures (Imbriquées) ---
        StructField("nutriments", nutriments_schema, True)
    ])

print("✅ Schéma JSONL défini.")

✅ Schéma JSONL défini.


In [4]:
# Adapte le nom du fichier si nécessaire
input_file = "openfoodfacts-products.jsonl"
raw_path = str(project_root / "data" / "raw" / input_file)

print(f"📂 Préparation de la lecture : {raw_path}")

try:
    df_raw = spark.read \
        .schema(get_jsonl_schema()) \
        .json(raw_path)

    print("✅ Lecture configurée (Lazy). Le chargement réel se fera après le sampling.")


    count = df_raw.count()
    print(f"📊 Nombre de produits ingérés : {count:,}")

    df_raw.printSchema()

except Exception as e:
    print(f"❌ Erreur de lecture : {e}")

📂 Préparation de la lecture : /Users/cedricsanchez/Master1/Cours/integration_donnees_TP/data/raw/openfoodfacts-products.jsonl
✅ Lecture configurée (Lazy). Le chargement réel se fera après le sampling.


📊 Nombre de produits ingérés : 4,247,844
root
 |-- code: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- last_modified_t: long (nullable = true)
 |-- created_t: long (nullable = true)
 |-- brands: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- countries_tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutriscore_grade: string (nullable = true)
 |-- nova_group: integer (nullable = true)
 |-- ecoscore_grade: string (nullable = true)
 |-- nutriments: struct (nullable = true)
 |    |-- energy-kcal_100g: float (nullable = true)
 |    |-- sugars_100g: float (nullable = true)
 |    |-- salt_100g: float (nullable = true)
 |    |-- sodium_100g: float (nullable = true)
 |    |-- fiber_100g: float (nullable = true)
 |    |-- proteins_100g: float (nullable = true)



## 🧹 Étape 2 : Transformation "Silver" (Nettoyage & Typage)

Maintenant que les données brutes sont chargées, nous devons les rendre utilisables pour l'analyse et le SCD2.
Cette étape applique les règles de qualité demandées :

1.  **Typage Temporel :** Conversion des timestamps UNIX (`Long`) en vraies dates (`Timestamp`) pour `last_modified_t` et `created_t`.
2.  **Nettoyage Textuel :** Suppression des espaces superflus (`trim`) sur les codes-barres et noms.
3.  **Extraction des Nutriments :** Aplatissement de la structure imbriquée `nutriments` pour faciliter les requêtes SQL futures.
4.  **Gestion des Nulls :** Conversion sécurisée de `nova_group` (texte vers entier) et filtrage des produits sans code-barre.

In [5]:
from pyspark.sql.functions import col, trim, from_unixtime, to_timestamp, when

print("⏳ Démarrage du nettoyage Silver avec Sampling...")

# On ne garde que 10% des données (environ 400k lignes) pour sauver le disque dur.
# seed=42 permet d'avoir toujours les mêmes 10% si on relances.
df_sampled = df_raw.sample(withReplacement=False, fraction=0.1, seed=42)

df_silver = df_sampled \
    .select(
        # --- 1. Nettoyage des Clés & Textes ---
        trim(col("code")).alias("code"),
        trim(col("product_name")).alias("product_name"),

        # --- 2. Gestion Temporelle ---
        from_unixtime(col("last_modified_t")).cast("timestamp").alias("last_modified_ts"),
        from_unixtime(col("created_t")).cast("timestamp").alias("created_ts"),

        # --- 3. Normalisation des Dimensions ---
        col("countries_tags"),
        trim(col("brands")).alias("brands"),
        trim(col("categories")).alias("categories"),

        # --- 4. Qualité & Scores ---
        trim(col("nutriscore_grade")).alias("nutriscore_grade"),
        trim(col("ecoscore_grade")).alias("ecoscore_grade"),
        col("nova_group").cast("integer").alias("nova_group"),

        # --- 5. Nutriments ---
        col("nutriments.energy-kcal_100g").alias("energy_kcal_100g"),
        col("nutriments.sugars_100g").alias("sugars_100g"),
        col("nutriments.salt_100g").alias("salt_100g"),
        col("nutriments.proteins_100g").alias("proteins_100g")
    ) \
    .filter(col("code").isNotNull()) \
    .filter(col("code") != "")

# On met en cache ce petit échantillon
df_silver.cache()

count = df_silver.count()
print(f"✅ Nettoyage terminé sur l'échantillon. Produits restants : {count:,}")
print("(C'est normal d'en avoir moins, on a pris 10% volontairement !)")

display(df_silver.select("code", "last_modified_ts", "product_name").limit(5))

⏳ Démarrage du nettoyage Silver avec Sampling...


✅ Nettoyage terminé sur l'échantillon. Produits restants : 425,588
(C'est normal d'en avoir moins, on a pris 10% volontairement !)


DataFrame[code: string, last_modified_ts: timestamp, product_name: string]

## 🔐 Étape 3 : Fingerprinting (Préparation SCD2)

Pour gérer l'historique (SCD2) efficacement, nous ne pouvons pas comparer toutes les colonnes à chaque fois.
Nous allons générer un **Hash Technique (`row_hash`)** : une empreinte digitale unique basée sur les colonnes métier.

* **Stratégie :** On concatène toutes les colonnes importantes (Nom, Marque, Nutriscore, Sucre...) et on applique un hachage SHA-256.
* **Intérêt :** Si le hash change, cela signifie que le produit a été modifié. C'est ce qui déclenchera la création d'une nouvelle version dans le Datamart.

In [6]:
from pyspark.sql.functions import sha2, concat_ws, col

print("⏳ Calcul du Hash (Fingerprint) pour chaque produit...")

columns_to_hash = [
    "product_name", "brands", "categories", "countries_tags",
    "nutriscore_grade", "nova_group", "ecoscore_grade",
    "energy_kcal_100g", "sugars_100g", "salt_100g", "proteins_100g"
]

df_hashed = df_silver.withColumn(
    "row_hash",
    sha2(concat_ws("||", *[col(c) for c in columns_to_hash]), 256)
)

print("✅ Hashing terminé.")
display(df_hashed.select("code", "product_name", "row_hash").limit(5))

⏳ Calcul du Hash (Fingerprint) pour chaque produit...
✅ Hashing terminé.


DataFrame[code: string, product_name: string, row_hash: string]

## 🏭 Étape 4 : Initialisation du Datamart (DDL)

Avant de charger les données, nous devons créer la structure des tables dans MySQL.
Nous utilisons une connexion Python directe (hors Spark) pour définir précisément :
1.  **Les Clés Primaires (PK) :** `product_sk` (Auto-incrément) pour identifier unique une *version* de produit.
2.  **Les Index :** Sur `code` et `row_hash` pour que les recherches (Join/Upsert) soient instantanées.
3.  **Les Colonnes SCD2 :** `effective_from` (début), `effective_to` (fin), `is_current` (actif).

In [7]:
import mysql.connector

def init_datamart():
    print("⏳ Initialisation et Tuning MySQL...")

    conn = mysql.connector.connect(
        host="localhost",
        port=3306,
        user=MYSQL_CONFIG["user"],
        password=MYSQL_CONFIG["password"],
        database="openfoodfacts"
    )
    cursor = conn.cursor()

    # --- TUNING ---
    cursor.execute("SET GLOBAL max_allowed_packet=67108864")

    # --- NETTOYAGE ---
    cursor.execute("DROP TABLE IF EXISTS fact_nutrition_snapshot")
    cursor.execute("DROP TABLE IF EXISTS dim_product")

    # --- DIMENSION (Pas de changement) ---
    product_ddl = """
    CREATE TABLE dim_product (
        product_sk INT AUTO_INCREMENT PRIMARY KEY,
        code VARCHAR(255) NOT NULL,
        product_name TEXT,
        brands TEXT,
        categories TEXT,
        row_hash CHAR(64) NOT NULL,
        effective_from DATETIME,
        effective_to DATETIME,
        is_current BOOLEAN,
        INDEX idx_code (code),
        INDEX idx_hash (row_hash)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    cursor.execute(product_ddl)
    print("   - Table 'dim_product' créée.")

    fact_ddl = """
    CREATE TABLE fact_nutrition_snapshot (
        fact_sk INT AUTO_INCREMENT PRIMARY KEY,
        product_sk INT NOT NULL,
        date_sk INT NOT NULL,

        nutriscore_grade VARCHAR(50),
        ecoscore_grade VARCHAR(50),
        nova_group INT,
        energy_kcal_100g FLOAT,
        sugars_100g FLOAT,
        salt_100g FLOAT,
        proteins_100g FLOAT,

        FOREIGN KEY (product_sk) REFERENCES dim_product(product_sk)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    cursor.execute(fact_ddl)
    print("   - Table 'fact_nutrition_snapshot' créée (avec VARCHAR).")

    conn.close()
    print("✅ Datamart prêt.")

try:
    init_datamart()
except Exception as e:
    print(f"❌ Erreur MySQL : {e}")

⏳ Initialisation et Tuning MySQL...
❌ Erreur MySQL : name 'MYSQL_CONFIG' is not defined


## 🚚 Étape 5 : Chargement de la Dimension Produit (Initial Load)

Nous séparons les données en deux flux :
1.  **Dimension (`dim_product`) :** Contient les descriptions et l'historique.
2.  **Faits (`fact_nutrition_snapshot`) :** Contient les chiffres.

Ici, nous chargeons la dimension.
* **Transformation :** On ne garde que les colonnes descriptives.
* **Initialisation SCD2 :** Comme c'est le premier chargement, on fixe :
    * `effective_from` = La date de modification du produit (`last_modified_ts`).
    * `effective_to` = '9999-12-31' (Date infinie = produit actif).
    * `is_current` = True.
* **Écriture JDBC :** On pousse vers MySQL en mode `append`.

In [8]:
from pyspark.sql.functions import lit
from etl.shared.config import MYSQL_CONFIG

print("⏳ Préparation de la dimension Produit...")

# Sélection finale
df_dim_product_init = df_hashed.select(
    col("code"),
    col("product_name"),
    col("brands"),
    col("categories"),
    col("row_hash"),
    col("last_modified_ts").alias("effective_from"),
    lit("9999-12-31 23:59:59").cast("timestamp").alias("effective_to"),
    lit(True).alias("is_current")
)

# Config JDBC optimisée
jdbc_url = MYSQL_CONFIG["url"]
jdbc_props = {
    "user": MYSQL_CONFIG["user"],
    "password": MYSQL_CONFIG["password"],
    "driver": MYSQL_CONFIG["driver"],

    # On réduit à 1000 pour ménager le réseau et la mémoire
    "batchsize": "1000"
}

print("🚀 Écriture dans MySQL (dim_product)...")

try:
    df_dim_product_init.write \
        .jdbc(url=jdbc_url, table="dim_product", mode="append", properties=jdbc_props)

    print("✅ Chargement terminé avec succès !")

except Exception as e:
    print(f"❌ Erreur d'écriture : {e}")

⏳ Préparation de la dimension Produit...
🚀 Écriture dans MySQL (dim_product)...
❌ Erreur d'écriture : An error occurred while calling o212.jdbc.
: com.mysql.cj.jdbc.exceptions.CommunicationsException: Communications link failure

The last packet sent successfully to the server was 0 milliseconds ago. The driver has not received any packets from the server.
	at com.mysql.cj.jdbc.exceptions.SQLError.createCommunicationsException(SQLError.java:165)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:55)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:837)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:420)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:238)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:180)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:50)
	at org.apach

## 📊 Étape 6 : Chargement de la Table de Faits (Fact Table)

C'est l'étape finale. Nous devons charger les mesures (sucre, sel, nutriscore...) dans `fact_nutrition_snapshot`.
**Le défi :** La table de faits a besoin de la clé étrangère `product_sk`. Or, cette clé a été générée par MySQL (Auto-increment) à l'étape précédente. Spark ne la connait pas.

**Stratégie :**
1.  **Lecture (Lookup) :** On lit la table `dim_product` depuis MySQL pour récupérer le couple `(code, product_sk)`.
2.  **Jointure :** On joint ces IDs avec notre DataFrame Spark (`df_hashed`) sur le code-barre.
3.  **Calcul Date Key :** On transforme la date en entier `YYYYMMDD` (ex: `20230520`) pour la clé de temps `date_sk`.
4.  **Écriture :** On insère les lignes dans `fact_nutrition_snapshot`.

In [9]:
from pyspark.sql.functions import date_format

print("⏳ Chargement des Faits : Récupération des IDs MySQL...")

# 1. On relit la dimension depuis MySQL pour avoir les product_sk générés
df_dim_mysql = spark.read \
    .format("jdbc") \
    .option("url", MYSQL_CONFIG["url"]) \
    .option("dbtable", "dim_product") \
    .option("user", MYSQL_CONFIG["user"]) \
    .option("password", MYSQL_CONFIG["password"]) \
    .option("driver", MYSQL_CONFIG["driver"]) \
    .load() \
    .select("product_sk", "code")

# 2. Jointure : Spark (Données) + MySQL (IDs)
df_facts = df_hashed.join(df_dim_mysql, on="code", how="inner")

# 3. Préparation finale des colonnes de la Fact Table
df_facts_final = df_facts.select(
    col("product_sk"), # La clé étrangère récupérée de MySQL

    # Création d'une clé de date simple (YYYYMMDD) basée sur la date de modif
    date_format(col("last_modified_ts"), "yyyyMMdd").cast("integer").alias("date_sk"),

    # Les Mesures
    col("nutriscore_grade"),
    col("ecoscore_grade"),
    col("nova_group"),
    col("energy_kcal_100g"),
    col("sugars_100g"),
    col("salt_100g"),
    col("proteins_100g")
)

print(f"📦 Prêt à charger {df_facts_final.count():,} lignes de faits.")

# 4. Écriture dans MySQL
print("🚀 Écriture dans 'fact_nutrition_snapshot'...")
try:
    df_facts_final.write \
        .jdbc(
            url=MYSQL_CONFIG["url"],
            table="fact_nutrition_snapshot",
            mode="append",
            properties=jdbc_props
        )
    print("✅ TERMINE ! Le Datamart est complet (Dimensions + Faits).")

except Exception as e:
    print(f"❌ Erreur écriture Faits : {e}")

⏳ Chargement des Faits : Récupération des IDs MySQL...


Py4JJavaError: An error occurred while calling o225.load.
: com.mysql.cj.jdbc.exceptions.CommunicationsException: Communications link failure

The last packet sent successfully to the server was 0 milliseconds ago. The driver has not received any packets from the server.
	at com.mysql.cj.jdbc.exceptions.SQLError.createCommunicationsException(SQLError.java:165)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:55)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:837)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:420)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:238)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:180)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:50)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(Connect